In [1]:
import pickle
import pandas as pd


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding, Flatten
from tensorflow.keras.optimizers import Adam

random_state=42

In [2]:
def precision_at_k(true_labels, predicted_labels, k=3):
    '''
    Рассчитывает precision_at_k
    # Пример использования
    true_labels = [1, 2, 3, 4, 5]  # Истинные метки
    predicted_labels = [1, 2, 6, 4, 7]  # Предсказанные рекомендации

    # Рассчитаем Precision@K для K=3
    k = 3
    precision = precision_at_k(true_labels, predicted_labels, k)
    print(f'Precision@{k}: {precision}')
    '''
    assert len(true_labels) == len(predicted_labels), "Длины списков должны совпадать"
    # Отбираем K предсказанных элементов
    pred_labels_at_k = predicted_labels[:k]
    # Количество релевантных элементов в K предсказанных
    relevant_at_k = sum([1 for label in pred_labels_at_k if label in true_labels])
    # Precision@K
    precision = relevant_at_k / k
    return precision

In [3]:
events = pd.read_pickle('lightfm/data/events_1.pkl')
test = pd.read_pickle('lightfm/data/test_time_1.pkl')
train = pd.read_pickle('lightfm/data/train_time_1.pkl')
test = test.sample(frac=0.5, random_state=random_state)

event_type = {
            'view': 0,
            'addtocart':0,
            'transaction': 10,
            }

test['event'] = test['event'].map(event_type)
train['event'] = train['event'].map(event_type)

In [4]:
n_items = max(events["itemid"].max(),events["itemid"].nunique())+1
n_users =  max(events["visitorid"].max(),events["visitorid"].nunique())+1

В первую очередь нам необходимо создать эмбеддинги для items и пользователей. Создаём эмбеддинги для items:  
Делаем то же самое для пользователей:

In [5]:
items_input = Input(shape=[1], name="items-Input")
items_embedding = Embedding(input_dim=n_items,
                            output_dim=10,
                            name="items-Embedding")(items_input)
items_vec = Flatten(name="Flatten-items")(items_embedding)


user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(input_dim=n_users,
                           output_dim= 10, 
                           name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для items, так и для пользователей, нам необходимо соединить их:

In [6]:
conc = Concatenate()([items_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [7]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(64, activation='relu')(fc1)
fc3 = Dense(32, activation='relu')(fc2)
fc4 = Dense(16, activation='relu')(fc3)
out = Dense(1)(fc4)

Собираем модель — передаём входные данные для items и пользователей, а также архитектуру нейронной сети:

In [8]:
model = Model([user_input, items_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку:

In [9]:
model.compile(optimizer=Adam(), loss='mape')

In [10]:
history = model.fit([train.visitorid, train.itemid], train.event, epochs=20, verbose=1)

Epoch 1/20


c:\Users\olgas\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User-Input', 'items-Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1066s 97ms/step - loss: 160684.3281
Epoch 2/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1101s 101ms/step - loss: 113938.0469
Epoch 3/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1060s 97ms/step - loss: 107244.0781
Epoch 4/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1034s 94ms/step - loss: 117174.2500
Epoch 5/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1037s 95ms/step - loss: 122605.8438
Epoch 6/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1038s 95ms/step - loss: 108020.1562
Epoch 7/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1039s 95ms/step - loss: 116967.3047
Epoch 8/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1042s 95ms/step - loss: 112638.7969
Epoch 9/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1045s 95ms/step - loss: 109929.8750
Epoch 10/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1047s 96ms/step - loss: 118239.9531
Epoch 11/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1049s 96ms/step - loss: 107511.5859
Epoch 12/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 1052s 96ms/step - loss: 120522.3359
Epoch 13/20
10948/10948 ━━━━━━━━━━━━━━━━━━━━ 10

В параметр эпох передаём значение 20: у нас будет реализовано 20 эпох — 20 обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [11]:
history.history

{'loss': [122469.328125,
  112585.9140625,
  108290.7890625,
  116586.6953125,
  114814.28125,
  108397.078125,
  115008.40625,
  111285.2421875,
  111226.015625,
  112355.3359375,
  108276.6640625,
  114542.8671875,
  111238.6171875,
  107071.0390625,
  109211.7421875,
  109998.6015625,
  107759.96875,
  114076.4609375,
  115632.6171875,
  110605.9375]}

In [12]:
model.evaluate([test.visitorid, test.itemid], test.event)

2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 718us/step - loss: 2263.7417


2263.808349609375

In [13]:
# Сохранение модели
model.save('my_model.keras')
print('Model saved as my_model.keras')

Model saved as my_model.keras


In [14]:
# Загрузка модели
model = load_model('my_model.keras')
print('Model loaded from my_model.keras')

# Рекомпиляция модели
model.compile(optimizer=Adam(), loss='mape')

Model loaded from my_model.keras


In [15]:
# Сделай предсказание
predictions = model.predict([test['visitorid'], test['itemid']])

print(predictions)


2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 957us/step
[[-2.3086177e-06]
 [-2.3086177e-06]
 [-2.3086177e-06]
 ...
 [-2.3086177e-06]
 [-2.3086177e-06]
 [-2.3086177e-06]]


In [16]:
predictions = predictions.flatten().tolist()

In [17]:
# Вычисление precision@k
k = 3
precision = precision_at_k(test['event'], predictions, k)

print(f'Precision@{k}: {precision}')


Precision@3: 0.0


In [18]:
for i in range(len(predictions )):
    if predictions[i]>2:
        print(predictions[i],i)

Как-то все очень плохо!!!  С учетом, как долго обучается, не буду тратить на это время и пытяться улучшить модель.